# Feature Engineering: Presidential Election Contributions

Let's clean up the presidential contributions dataset by feature engineering.

In [ ]:
%matplotlib inline
import time
from pyspark.sql.functions import isnan, when, count, col, split, trim, countDistinct, abs 
from pyspark.sql.types import IntegerType
import pyspark.sql.functions

print('Spark UI running on http://YOURIPADDRESS:' + sc.uiWebUrl.split(':')[2])

## About the data
In US contributions to elections are public record published by Federal Election Commission (FEC).   
Here we have a sample of 10k for 2016 contributions.  

Optionally, To download the full dataset, you can execute the **download-data.sh** 

## Step 1: Load the data

In [ ]:
%%time
t1 = time.perf_counter()
#Load presidential contrib data
contribs = spark.read.csv("/data/presidential_election_contribs/2016/2016-100k.csv.gz",\
                          header=True, inferSchema=True)
t2 = time.perf_counter()
print("read {:,} records in {:,.2f} ms".format(contribs.count(), (t2-t1)*1000))



In [ ]:
## TODO: printSchema
contribs.???

## TODO : how many records do we have?
## Hint : count
print (contribs.???())

## TODO : see some sample data
## HInt : show()
contribs.???()


## Step 2 : Select fields to analyze
The dataset has quite a few fields.  Let's start with some obvious ones.  
YOu can always add more fields later.

In [ ]:
## TODO : extract the following fields
##    - Candidate Name : 'CAND_NM
##    - Contributor Name : 'CONTBR_NM'
##    - Contributor Occupation : ????
##    - COntribution Amount : ????
##    - contribution zip code : ???

contribs2 = contribs.select([ 'CAND_NM', 'CONTBR_NM', '???', '???', '???'])
print(contribs2.count())
contribs2.show(20, False)

## Step 3: Register SQL tables
For ease of analytics let's also register these as SQL temp tables

In [ ]:
## Hint : use 'createOrReplaceTempView' function
contribs.???("contribs")
contribs2.???("contribs2")

## list tables
## Hint : Use function in spark.catalog.
## Use tab completion
spark.catalog.???()

## Step 4: Negative Contributions ??!??

As a cleanup, let us check for contribution amounts less than zero, and fix if necessary.

In [ ]:
# Contribution receipt less than zero?
## TODO : filter
## HINT : filter ('column name for amount' < 0)
#negative_contrib = contrib2.???(' condition ')
negative_contrib = contribs2.filter('???')

## TODO : how many negative contributions do we have?
## Hint : count
print(negative_contrib.count())

## TODO : print some sample data
## Hint : show()
negative_contrib.show(10, False)

## TODO : Now try the query in SQL
#neg = spark.sql("your sql query to find negative contribution goes here")
#neg.show()

## Step 5 : Extract data that only has positive contribution amount

In [ ]:
## TODO: 
##     extract only positive  contribs
pos_contribs = contribs2.filter('???')

print(pos_contribs.count())

In [ ]:
## TODO : in SQL
pos_contribs2 = spark.sql("query for possitive contributions go here")
pos_contribs2.show()

## now register this as a new table
pos_contribs2.createOrReplaceTempView("pos_contribs")

## TODO : count how many postivive contribs 
spark.sql("???").show()

## Step 6: Split up name into first name and last name
This is an example on how to create new columns

In [ ]:
split_col = split(contribs2['CONTBR_NM'], ',')
first_last_name = contribs2.withColumn('LASTNAME', trim(split_col.getItem(0)))
first_last_name = first_last_name.withColumn('FIRSTNAME', trim(split_col.getItem(1)))

first_last_name.show()

## register as sql table
first_last_name.createOrReplaceTempView("pos_contribs")

## Step 7: Analyze data
We can use Dataframe DSL language or SQL queries to perform analysis.  
Practice both.   
We will give you hints along the way.

### 7.1 - Count contributions per candidate

In [ ]:
## TODO : What is the breakdown by candidate name?
## Hint : groupBy("name").count()

pos_contribs.groupBy("???").count().show()


## TODO : try SQL queries

s="""
select CAND_NM, ????
????

"""

spark.sql(s).show()


## TODO : sort the result by contrib total



### 7.2 - Calculate AVG, MIN, MAX contributions per candidate
For this we can easily use SQL

In [ ]:
## TODO 

s="""
select CAND_NM, AVG(???)  as avg_contrib , MAX(???) as max_contrib
from pos_contribs
group by ??? order by ???  DESC
"""

avg_per_candidate = spark.sql("")
avg_per_candidate.show(20, False)

### 7.3 - Calculate AVG contribution by Occupation
If you are political consultants, which people you might try to solicit money?

In [ ]:
## TODO 
#avg_per_occupation = spark.sql("")

### 7.4 - Find Zipcodes that give most money

In [ ]:
## TODO : Use a sql query
## Hint : you want to calcluate average contribution per zip
## also calculate number of contribs per zipcode

s="""
write your sql here
"""

spark.sql(s).show()

In [ ]:
## Hint : to extract 5 digit zipcode 
## you can use : substring(CONTBR_ZIP, 0, 5) as zip

## Class Discussion
Now that we have done some feature engineering, waht other attributes we can extract and analyze?